<a href="https://colab.research.google.com/github/BiancadeFrancisco/SistemaRecomendacao_Trending/blob/main/C%C3%B3pia_de_SISTEMA_DE_RECOMENDA%C3%87%C3%83O_TRENDING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recomendação Trending

<!-- Este notebook contém 2 exemplos de recomendação Top N:

- **Top N consumidos**: os N itens mais consumidos pelos usuários
- **Top N avaliados**: os N itens melhores avaliados pelos usuários -->

O dataset a ser utilizado será o [MovieLens](https://grouplens.org/datasets/movielens/), cuja análise exploratória foi feita no exemplo prático do módulo **Introdução aos Sistemas de Recomendação**.

- Trending: itens que estão em ascensão.

In [ ]:
import os
import re
import sys
import pandas as pd
from datetime import datetime
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#007efd', '#FFC000', '#303030'])

## Arquivo de avaliações

Upload file `ratings.parquet`

In [ ]:
%%time
_ = files.upload()

Saving ratings.parquet to ratings.parquet
CPU times: user 669 ms, sys: 156 ms, total: 825 ms
Wall time: 50.5 s


In [ ]:
def convert_timestamp_to_date(timestamp:int):
    return datetime.fromtimestamp(timestamp).date()

df_ratings = pd.read_parquet('ratings.parquet')
df_ratings['date'] = df_ratings['timestamp'].apply(convert_timestamp_to_date)
df_ratings.tail()

,user_id,item_id,rating,timestamp,date
1000204,6040,1091,1,956716541,2000-04-26
1000205,6040,1094,5,956704887,2000-04-25
1000206,6040,562,5,956704746,2000-04-25
1000207,6040,1096,4,956715648,2000-04-26
1000208,6040,1097,4,956715569,2000-04-26


In [ ]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1000209 non-null  int64 
 1   item_id    1000209 non-null  int64 
 2   rating     1000209 non-null  int64 
 3   timestamp  1000209 non-null  int64 
 4   date       1000209 non-null  object
dtypes: int64(4), object(1)
memory usage: 38.2+ MB


## Arquivo de metadados dos itens

Upload file `movies.parquet`

In [ ]:
%%time
_ = files.upload()

Saving movies.parquet to movies (1).parquet
CPU times: user 78.4 ms, sys: 11.9 ms, total: 90.3 ms
Wall time: 6.11 s


In [ ]:
def extract_year_from_title(title:str, regex='(\d{4})'):
    match = re.search(regex, title)
    return None if match is None else match.group()

def convert_genres_to_list(genres:str, separator='|'):
    return genres.split(separator)

df_items = pd.read_parquet('movies.parquet')
df_items['genres'] = df_items['genres'].apply(convert_genres_to_list)
df_items['year'] = df_items['title'].apply(extract_year_from_title)
df_items.tail()

,item_id,title,genres,year
3878,3948,Meet the Parents (2000),[Comedy],2000
3879,3949,Requiem for a Dream (2000),[Drama],2000
3880,3950,Tigerland (2000),[Drama],2000
3881,3951,Two Family House (2000),[Drama],2000
3882,3952,"Contender, The (2000)","[Drama, Thriller]",2000


# Cálculo do Trending

A recomendação _trending_ busca apresentar os itens que tiveram maior _lift_ de consumo em uma determinada janela de tempo. Em termos matemáticos, o _lift_ pode ser definido como:

$$lift = \frac{consumoJanelaAtual-consumoJanelaAnterior}{consumoJanelaAnterior}$$

Neste notebook, utilizaremos como janela de tempo o mês de consumo do item. A função abaixo nos auxilia a definir uma coluna `window` que será utilizada na lógica do algoritmo.

In [ ]:
# JANELA = PERIODO DE TEMPO EM QUE ESTAMOS CONSIDERANDO PARA CONTABILIZAR AUMENTO OU DIMINUIÇÃO DE CONSUMO
# 1º: DEFINIR O QUE É A JANELA. (estou vendo um trending diário, semanal, mensal?)

# IREMOS UTILIZAR COMO JANELA, O ANO E O MÊS DA AVALIAÇÃO DOS USUÁRIOS!

def extract_year_month(date):
    return '{:04d}-{:02d}'.format(date.year, date.month)

# PARA DEFINIR A FUNÇÃO: NOME DA FUNÇÃO(O QUE ELA IRÁ RECEBER), RETORNANDO OS RESULTADOS QUE QUERO

df_ratings['window'] = df_ratings['date'].apply(extract_year_month) # CRIEI UMA COLUNA CHAMADA WINDOW QUE SERÁ IGUAL A COLUNA DATA, PORÉM APLICANDO A FUNÇÃO DESCRITA ACIMA
df_ratings.tail()

,user_id,item_id,rating,timestamp,date,window
1000204,6040,1091,1,956716541,2000-04-26,2000-04
1000205,6040,1094,5,956704887,2000-04-25,2000-04
1000206,6040,562,5,956704746,2000-04-25,2000-04
1000207,6040,1096,4,956715648,2000-04-26,2000-04
1000208,6040,1097,4,956715569,2000-04-26,2000-04


## Consumo por janela temporal

In [ ]:
# PRECISO AVALIAR O QUANTO TEVE DE CONSUMO  PARA CADA ITEM, COMPARANDO NA JANELA ATUAL COM A ANTERIOR, SE HOUVE AUMENTO OU DIMINUIÇÃO DE CONSUMO


df_window_consumptions = (
    df_ratings
    .groupby(['item_id', 'window']) # AGRUPAR POR ITEM E JANELA
    .agg({'user_id': 'count'}) # PRECISO AGREGAR, USANDO UM DICIONARIO, CONTAR O NUMERO DE USUARIOS (VISUALIZAÇÕES)
)
df_window_consumptions

user_id
item_id window          
1       2000-04       17
        2000-05      165
        2000-06      128
        2000-07      203
        2000-08      386
...                  ...
3952    2002-09        1
        2002-11        1
        2002-12        3
        2003-01        2
        2003-02        1

[65640 rows x 1 columns]

In [ ]:
# CÓDIGO COMPLETO PARA ANÁLISE

df_window_consumptions = (
    df_ratings
    .groupby(['item_id', 'window']) # AGRUPAR POR ITEM E JANELA
    .agg({'user_id': 'count'}) # PRECISO AGREGAR, USANDO UM DICIONARIO, ONTAR O NUMERO DE USUARIOS
    .reset_index()
    .rename({'user_id': 'count'}, axis=1) # RENOMEAR A COLUNA USER_ID PARA COUNT
    .sort_values(by=['item_id', 'window']) # ORDENAR POR ITEM E JANELA
)
df_window_consumptions

,item_id,window,count
0,1,2000-04,17
1,1,2000-05,165
2,1,2000-06,128
3,1,2000-07,203
4,1,2000-08,386
...,...,...,...
65635,3952,2002-09,1
65636,3952,2002-11,1
65637,3952,2002-12,3
65638,3952,2003-01,2


## Shift temporal

Para realizar operações entre o valor atual e o anterior de uma janela temporal podemos **deslocar** (_shift_) os valores de uma coluna a partir de um agrupamento. Exemplo:

| Agrupamento | Valor | Shift Valor |
|--------|-------|-------------|
| A      | A1    | N/A         |
| A      | A2    | A1          |
| B      | B1    | N/A         |
| B      | B2    | B1          |
| B      | B3    | B2          |

In [ ]:
# PARA CALCULAR O LIFT, PRECISO DA DIFERENÇA ENTRE AS JANELAS ATUAIS E ANTERIORES, PORTANTO NECESISTA CRIAR UMA COLUNA COM ESSA DIFERENÇA ENTRE AS JANELAS

df_window_consumptions.sort_values(by=['item_id', 'window'], inplace=True)

df_window_consumptions['count_previous'] = ( #NOME DA NOVA COLUNA
    df_window_consumptions #DATAFRAME QUE IREMOS UTILIZAR
    .groupby(['item_id'])['count'] #O AGRUPAMENTO QUE IRÁ SOFRER ESSA CONTABILIZAÇÃO
    .shift(1) # SHIFT EM UMA LINHA (ENTRE UMA LINHA E OUTRA)
)
df_window_consumptions

,item_id,window,count,count_previous
0,1,2000-04,17,NaN
1,1,2000-05,165,17.0
2,1,2000-06,128,165.0
3,1,2000-07,203,128.0
4,1,2000-08,386,203.0
...,...,...,...,...
65635,3952,2002-09,1,2.0
65636,3952,2002-11,1,1.0
65637,3952,2002-12,3,1.0
65638,3952,2003-01,2,3.0


## Lift

Implementando a seguinte fórmula:
$$lift = \frac{contagemJanelaAtual-contagemJanelaAnterior}{contagemJanelaAnterior}$$

In [ ]:
df_window_consumptions['lift'] = (df_window_consumptions['count'] - df_window_consumptions['count_previous'])/df_window_consumptions['count_previous']
df_window_consumptions

# EXEMPLO, O ITEM 1 TEVE UM LIFT DE 8,7 VISUALIZAÇÕES ENTRE O MÊS 05/2000 E O MÊS 04/2000

,item_id,window,count,count_previous,lift
0,1,2000-04,17,NaN,NaN
1,1,2000-05,165,17.0,8.705882
2,1,2000-06,128,165.0,-0.224242
3,1,2000-07,203,128.0,0.585938
4,1,2000-08,386,203.0,0.901478
...,...,...,...,...,...
65635,3952,2002-09,1,2.0,-0.500000
65636,3952,2002-11,1,1.0,0.000000
65637,3952,2002-12,3,1.0,2.000000
65638,3952,2003-01,2,3.0,-0.333333


## Especificando janela de recomendação

Para a recomendação precisamos de uma janela de referência. Por exemplo:

- Trending na janela atual?
- Trending na janela anterior?
- Trending em uma janela específica?

Definida a janela de referência, utilizamos o valor do _lift_ como _score_ do item e ordenamos pelo _score_.

In [ ]:
prediction_window = '2003-01'
(
    df_window_consumptions
    .query('window == @prediction_window')
    .rename({'lift': 'score'}, axis=1)
    .sort_values(by='score', ascending=False)
)

,item_id,window,count,count_previous,score
32959,2011,2003-01,7,1.0,6.00
42100,2502,2003-01,7,1.0,6.00
782,32,2003-01,6,1.0,5.00
64914,3897,2003-01,6,1.0,5.00
25525,1527,2003-01,5,1.0,4.00
...,...,...,...,...,...
64221,3847,2003-01,1,4.0,-0.75
21114,1266,2003-01,1,5.0,-0.80
18836,1179,2003-01,1,5.0,-0.80
18426,1136,2003-01,1,5.0,-0.80


# Recomendando Itens Trending

Por fim, juntamos toda a lógica descrita até então na função `recommend_trending_n` abaixo para recomendar os N items que estão em maior ascensão.

In [ ]:
def recommend_trending_n(ratings:pd.DataFrame, n:int, prediction_window:str=None, min_evaluations:int=None) -> pd.DataFrame:
#NOME DA FUNÇÃO (recommend_trending_n), VAI RECEBER: ratings:pd.DataFrame (dataframe), n:int (numero de itens), prediction_window:str=None, min_evaluations:int=None (um filtro para mostrar apenas valores mínimos considerados relevantes).

    prediction_window = max(ratings['window']) if prediction_window is None else prediction_window

    ratings = ratings[['item_id', 'window', 'user_id']]
    # Calculo de janela
    df_window_consumptions = (
        ratings
        .groupby(['item_id', 'window'])['user_id']
        .count()
        .reset_index()
        .rename({'user_id': 'count'}, axis=1)
        .sort_values(by=['item_id', 'window'])
    )

    # Shift temporal
    df_window_consumptions['count_previous'] = (
        df_window_consumptions
        .groupby(['item_id'])['count']
        .shift(1)
    )

    # Calculo do lift
    df_window_consumptions['lift'] = (df_window_consumptions['count'] - df_window_consumptions['count_previous'])/df_window_consumptions['count_previous']

    # Selecao de janela
    recommendations = (
      df_window_consumptions
      .query('window == @prediction_window')
      .rename({'lift': 'score'}, axis=1)
      .sort_values(by='score', ascending=False)
    )

    if min_evaluations is not None:
        recommendations = recommendations.query('count_previous >= @min_evaluations') # filtro de itens com valores mínimos considerados

    return recommendations.head(n)

df_trending = recommend_trending_n(df_ratings, n=10, prediction_window='2002-12')
df_trending.merge(df_items, on='item_id', how='inner')

# exemplo, o filme Tomorrow Never Dies (1997), da janela 12/2002 teve na janela anterior 1 avaliação e na janela seguinte 7 avaliações, trazendo um score de 6 (cresceu 6x)

,item_id,window,count,count_previous,score,title,genres,year
0,1722,2002-12,7,1.0,6.0,Tomorrow Never Dies (1997),"[Action, Romance, Thriller]",1997
1,595,2002-12,7,1.0,6.0,Beauty and the Beast (1991),"[Animation, Children's, Musical]",1991
2,3503,2002-12,6,1.0,5.0,Solaris (Solyaris) (1972),"[Drama, Sci-Fi]",1972
3,3639,2002-12,6,1.0,5.0,"Man with the Golden Gun, The (1974)",[Action],1974
4,2990,2002-12,6,1.0,5.0,Licence to Kill (1989),[Action],1989
5,1266,2002-12,5,1.0,4.0,Unforgiven (1992),[Western],1992
6,1179,2002-12,5,1.0,4.0,"Grifters, The (1990)","[Crime, Drama, Film-Noir]",1990
7,3882,2002-12,5,1.0,4.0,Bring It On (2000),[Comedy],2000
8,2966,2002-12,4,1.0,3.0,"Straight Story, The (1999)",[Drama],1999
9,2942,2002-12,7,2.0,2.5,Flashdance (1983),"[Drama, Romance]",1983


Note que alguns itens tiveram _lift_/_score_ altos, porém **poucos usuários consumiram estes itens**.

Para evitar estes comportamentos, podemos estabelecer um limite mínimo de consumo para que um item seja recomendável.

In [ ]:
prediction_window = '2002-12' # janela filtrada
min_evaluations = 2 # filtrando numero mínimo de avaliações
n = 10 # 10 itens

df_trending = recommend_trending_n(
    df_ratings,
    n=n,
    prediction_window=prediction_window,
    min_evaluations=min_evaluations
)
df_trending.merge(df_items, on='item_id', how='inner')

,item_id,window,count,count_previous,score,title,genres,year
0,2942,2002-12,7,2.0,2.500000,Flashdance (1983),"[Drama, Romance]",1983
1,3791,2002-12,10,3.0,2.333333,Footloose (1984),[Drama],1984
2,2926,2002-12,6,2.0,2.000000,Hairspray (1988),"[Comedy, Drama]",1988
3,3635,2002-12,6,2.0,2.000000,"Spy Who Loved Me, The (1977)",[Action],1977
4,1032,2002-12,5,2.0,1.500000,Alice in Wonderland (1951),"[Animation, Children's, Musical]",1951
5,1097,2002-12,7,3.0,1.333333,E.T. the Extra-Terrestrial (1982),"[Children's, Drama, Fantasy, Sci-Fi]",1982
6,2054,2002-12,6,3.0,1.000000,"Honey, I Shrunk the Kids (1989)","[Adventure, Children's, Comedy, Fantasy, Sci-Fi]",1989
7,1375,2002-12,4,2.0,1.000000,Star Trek III: The Search for Spock (1984),"[Action, Adventure, Sci-Fi]",1984
8,1258,2002-12,4,2.0,1.000000,"Shining, The (1980)",[Horror],1980
9,1517,2002-12,4,2.0,1.000000,Austin Powers: International Man of Mystery (1...,[Comedy],1997


# Seleção de Acervo

Podemos selecionar itens específicos a serem recomendados filtrando o dataset de avaliações.

In [ ]:
genre = "Children's"
item_ids = df_items[df_items['genres'].apply(lambda x: genre in x)]['item_id']
# lambda x: genre in x = vai passar linha por linha do dataset e ver gênero por gênero, procurando itens que contenham o gênero "Children's"

df_ratings_filtered = df_ratings[df_ratings['item_id'].isin(item_ids)] # dentro da lista de itens = isin(item_ids)
df_ratings_filtered.tail()

,user_id,item_id,rating,timestamp,date,window
999888,6040,919,5,956704191,2000-04-25,2000-04
1000014,6040,34,4,956704584,2000-04-25,2000-04
1000153,6040,2384,4,956703954,2000-04-25,2000-04
1000191,6040,3751,4,964828782,2000-07-28,2000-07
1000208,6040,1097,4,956715569,2000-04-26,2000-04


In [ ]:
df_trending = recommend_trending_n(df_ratings_filtered, n=10, prediction_window='2002-12', min_evaluations=2)
df_trending.merge(df_items, on='item_id', how='inner')

,item_id,window,count,count_previous,score,title,genres,year
0,1032,2002-12,5,2.0,1.500000,Alice in Wonderland (1951),"[Animation, Children's, Musical]",1951
1,1097,2002-12,7,3.0,1.333333,E.T. the Extra-Terrestrial (1982),"[Children's, Drama, Fantasy, Sci-Fi]",1982
2,2054,2002-12,6,3.0,1.000000,"Honey, I Shrunk the Kids (1989)","[Adventure, Children's, Comedy, Fantasy, Sci-Fi]",1989
3,317,2002-12,7,4.0,0.750000,"Santa Clause, The (1994)","[Children's, Comedy, Fantasy]",1994
4,596,2002-12,3,2.0,0.500000,Pinocchio (1940),"[Animation, Children's]",1940
5,1,2002-12,3,2.0,0.500000,Toy Story (1995),"[Animation, Children's, Comedy]",1995
6,594,2002-12,2,2.0,0.000000,Snow White and the Seven Dwarfs (1937),"[Animation, Children's, Musical]",1937
7,2090,2002-12,2,2.0,0.000000,"Rescuers, The (1977)","[Animation, Children's]",1977
8,2761,2002-12,2,2.0,0.000000,"Iron Giant, The (1999)","[Animation, Children's]",1999
9,1028,2002-12,2,2.0,0.000000,Mary Poppins (1964),"[Children's, Comedy, Musical]",1964
